In [1]:
%pip install ..

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=20459 sha256=52953d8ed485416eff235716b0c8b9808e012591ae8bdc7c5f0ad3f9e3a4b2a3
  Stored in directory: /tmp/pip-ephem-wheel-cache-z3uecpxh/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1


In [2]:
import torch 

device = torch.device("cuda:3")

In [3]:
from spuco.invariant_train import ERM
from spuco.models import MLP
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    train=True
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    train=False
)
testset.initialize()


In [4]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [5]:
model = MLP(3*28*28, len(classes)).to(device)

In [6]:
from torch.optim import SGD

model = MLP(3*28*28, len(classes)).to(device)
erm = ERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

In [7]:
erm.train()

Epoch 0: 100%|██████████| 938/938 [00:02<00:00, 405.25batch/s, accuracy=100.0%, loss=0.00738] 


In [8]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 100.0
Group (0, 1) Test Accuracy: 0.0
Group (0, 2) Test Accuracy: 0.0
Group (0, 3) Test Accuracy: 0.0
Group (0, 4) Test Accuracy: 0.0
Group (1, 0) Test Accuracy: 0.0
Group (1, 1) Test Accuracy: 100.0
Group (1, 2) Test Accuracy: 0.0
Group (1, 3) Test Accuracy: 0.0
Group (1, 4) Test Accuracy: 0.0
Group (2, 0) Test Accuracy: 0.0
Group (2, 1) Test Accuracy: 0.0
Group (2, 2) Test Accuracy: 100.0
Group (2, 3) Test Accuracy: 0.0
Group (2, 4) Test Accuracy: 0.0
Group (3, 0) Test Accuracy: 0.0
Group (3, 1) Test Accuracy: 0.0
Group (3, 2) Test Accuracy: 0.0
Group (3, 3) Test Accuracy: 100.0
Group (3, 4) Test Accuracy: 0.0
Group (4, 0) Test Accuracy: 0.0
Group (4, 1) Test Accuracy: 0.0
Group (4, 2) Test Accuracy: 0.0
Group (4, 3) Test Accuracy: 0.0
Group (4, 4) Test Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 100.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 100.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 100.0}

In [9]:
evaluator.worst_group_accuracy

((0, 1), 0.0)

In [10]:
evaluator.average_accuracy

99.52666666666667

In [11]:
evaluator.evaluate_spurious_task()

100.0